<h3><font color="#00586D" size=4> Para que los ejemplos funcionen debes de crear una cola y poner el enlace en la segunda caja de comandos</font></h3>

In [1]:
import boto3

In [2]:
sqs = boto3.client('sqs',region_name='eu-west-3')
queue_url = "URL de tu cola"

<h3><font color="#00586D" size=4> Ejemplo 1: envío y recepcion de mensajes</font></h3>

In [5]:
response=sqs.send_message(
    QueueUrl=queue_url,
    MessageBody=(
        'Warhamer 40K es una franquicia de ciencia ficion oscura.'
    )
)
print(f'Enviado: {response["MessageId"]}')

Enviado: dfdfef27-c55b-4739-801e-36b8ba9a449d


In [6]:
response=sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=1
)

if 'Messages' in response:
    message=response['Messages'][0]
    receipt_handle=message['ReceiptHandle']
    
    sqs.delete_message(
        QueueUrl=queue_url,
        ReceiptHandle=receipt_handle
    )
    print(f'Recibido y borrado: {message["MessageId"]} {message["Body"]}')
else:
    print("No hay mensajes en la cola")

Recibido y borrado: dfdfef27-c55b-4739-801e-36b8ba9a449d Warhamer 40K es una franquicia de ciencia ficion oscura.


<h3><font color="#00586D" size=4> Ejemplo 2: tiempo de espera de visivilidad(visibility timeout)</font></h3>

In [8]:
response=sqs.send_message(
    QueueUrl=queue_url,
    MessageBody=(
        'Los platinos de los residents evils son un coñazo'
    )
)
print(f'Enviado: {response["MessageId"]}')

Enviado: ef395311-a88e-4813-9a4d-ceb224204142


In [11]:
response=sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=1
)

if 'Messages' in response:
    message=response['Messages'][0]
    receipt_handle=message['ReceiptHandle']
    print(f'Recibido {message["MessageId"]} {message["Body"]}')
else:
    print("No hay mensajes en la cola")

No hay mensajes en la cola


In [42]:
sqs.delete_message(
    QueueUrl=queue_url,
    ReceiptHandle=receipt_handle
)
print(f'Borrado: {message["MessageId"]} {message["Body"]}')

Borrado: 17261af5-df86-47a4-9191-982a028f9900 Los platinos de los resident evil son un coñazo


<h3><font color="#00586D" size=4> Ejemplo 3: retraso de entrega(delivery delay)</font></h3>

In [45]:
response=sqs.send_message(
    QueueUrl=queue_url,
    MessageBody=(
        'Me encanta el salseo y las funas'
    ),
    DelaySeconds=5
)
print(f'Enviado: {response["MessageId"]}')

Enviado: d4f6ac4a-27b0-4258-9fe5-c364d6b5f798


In [55]:
response=sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=1
)

if 'Messages' in response:
    message=response['Messages'][0]
    receipt_handle=message['ReceiptHandle']
    
    sqs.delete_message(
        QueueUrl=queue_url,
        ReceiptHandle=receipt_handle
    )
    print(f'Recibido y borrado: {message["MessageId"]} {message["Body"]}')
else:
    print("No hay mensajes en la cola")

No hay mensajes en la cola


<h3><font color="#00586D" size=4> Ejemplo 4: Sondeo largo (lorn polling)</font></h3>
<h6>Enviamos el mensaje, ejecutamos para que me diga a quien y que mensaje y en ese momento vuelvo a enviar el mensaje</h6>

In [70]:
import threading

def receive_and_delete():
    response=sqs.receive_message(
        QueueUrl=queue_url,
        MaxNumberOfMessages=1,
        WaitTimeSeconds=20
    )
    if 'Messages' in response:
        message=response['Messages'][0]
        receipt_handle=message['ReceiptHandle']
        sqs.delete_message(
            QueueUrl=queue_url,
            ReceiptHandle=receipt_handle
        )
        print(f'Recibido y borrado: {message["MessageId"]} {message["Body"]}')
    else:
        print("No hay mensajes en la cola")
x= threading.Thread(target=receive_and_delete)
x.start()

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_195/2600184032.py", line 4, in receive_and_delete
  File "/home/juanlopez/.local/lib/python3.8/site-packages/botocore/client.py", line 530, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/juanlopez/.local/lib/python3.8/site-packages/botocore/client.py", line 960, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.QueueDoesNotExist: An error occurred (AWS.SimpleQueueService.NonExistentQueue) when calling the ReceiveMessage operation: The specified queue does not exist for this wsdl version.


In [71]:
response=sqs.send_message(
    QueueUrl=queue_url,
    MessageBody=(
        'Prueba de sondeo largo'
    )
)

print(f'Enviado:{response["MessageId"]}')

QueueDoesNotExist: An error occurred (AWS.SimpleQueueService.NonExistentQueue) when calling the SendMessage operation: The specified queue does not exist for this wsdl version.